# 11.1 MCP Server 实现

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/ai-first-app/blob/main/demos/11-protocols/mcp_server.ipynb)

**预计 API 费用: ~$0**

本 Notebook 演示如何实现一个 MCP Server,并连接到 Claude Desktop。

In [ ]:
!pip install -q mcp

## 实验 1: 最简单的 MCP Server

实现一个计算器 Server,提供加减乘除四个工具。

In [ ]:
# calculator_server.py
from mcp.server import Server
from mcp.server.stdio import stdio_server
from mcp.types import Tool, TextContent

# 创建 MCP Server
app = Server("calculator-server")

# 定义工具列表
@app.list_tools()
async def list_tools() -> list[Tool]:
    return [
        Tool(
            name="add",
            description="计算两个数的和",
            inputSchema={
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "第一个数"},
                    "b": {"type": "number", "description": "第二个数"}
                },
                "required": ["a", "b"]
            }
        ),
        Tool(
            name="subtract",
            description="计算两个数的差",
            inputSchema={
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "被减数"},
                    "b": {"type": "number", "description": "减数"}
                },
                "required": ["a", "b"]
            }
        ),
        Tool(
            name="multiply",
            description="计算两个数的积",
            inputSchema={
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "第一个数"},
                    "b": {"type": "number", "description": "第二个数"}
                },
                "required": ["a", "b"]
            }
        ),
        Tool(
            name="divide",
            description="计算两个数的商",
            inputSchema={
                "type": "object",
                "properties": {
                    "a": {"type": "number", "description": "被除数"},
                    "b": {"type": "number", "description": "除数"}
                },
                "required": ["a", "b"]
            }
        )
    ]

# 实现工具调用
@app.call_tool()
async def call_tool(name: str, arguments: dict) -> list[TextContent]:
    a = arguments["a"]
    b = arguments["b"]
    
    if name == "add":
        result = a + b
    elif name == "subtract":
        result = a - b
    elif name == "multiply":
        result = a * b
    elif name == "divide":
        if b == 0:
            return [TextContent(type="text", text="错误: 除数不能为 0")]
        result = a / b
    else:
        return [TextContent(type="text", text=f"未知工具: {name}")]
    
    return [TextContent(type="text", text=str(result))]

# 启动 Server
async def main():
    async with stdio_server() as (read_stream, write_stream):
        await app.run(
            read_stream,
            write_stream,
            app.create_initialization_options()
        )

print("✅ MCP Server 代码准备完成")
print("\n保存为 calculator_server.py 后,可以通过以下方式启动:")
print("python calculator_server.py")

## 实验 2: 提供 Resources(资源)

除了工具,MCP Server 还可以提供 Resources,如文件、数据库内容等。

In [ ]:
from mcp.server import Server
from mcp.types import Resource, TextContent

app = Server("resource-server")

# 模拟数据
fake_docs = {
    "doc1": "这是第一个文档的内容",
    "doc2": "这是第二个文档的内容",
    "doc3": "这是第三个文档的内容"
}

# 列出可用资源
@app.list_resources()
async def list_resources() -> list[Resource]:
    return [
        Resource(
            uri=f"file:///{doc_id}",
            name=f"文档 {doc_id}",
            description=f"示例文档 {doc_id}",
            mimeType="text/plain"
        )
        for doc_id in fake_docs.keys()
    ]

# 读取资源
@app.read_resource()
async def read_resource(uri: str) -> str:
    doc_id = uri.replace("file:///", "")
    content = fake_docs.get(doc_id, "文档不存在")
    return content

print("✅ Resource Server 实现完成")
print("\nMCP Server 可以提供:")
print("  - Tools: 可执行的函数")
print("  - Resources: 静态或动态的内容")
print("  - Prompts: 提示词模板")

## 实验 3: 提供 Prompts(提示词模板)

MCP Server 还可以提供预定义的 Prompt 模板。

In [ ]:
from mcp.server import Server
from mcp.types import Prompt, PromptMessage

app = Server("prompt-server")

# 定义 Prompt 模板
@app.list_prompts()
async def list_prompts() -> list[Prompt]:
    return [
        Prompt(
            name="code_review",
            description="代码审查模板",
            arguments=[
                {
                    "name": "code",
                    "description": "要审查的代码",
                    "required": True
                },
                {
                    "name": "language",
                    "description": "编程语言",
                    "required": False
                }
            ]
        ),
        Prompt(
            name="write_blog",
            description="撰写技术博客模板",
            arguments=[
                {
                    "name": "topic",
                    "description": "博客主题",
                    "required": True
                }
            ]
        )
    ]

# 获取 Prompt 内容
@app.get_prompt()
async def get_prompt(name: str, arguments: dict) -> list[PromptMessage]:
    if name == "code_review":
        code = arguments.get("code", "")
        language = arguments.get("language", "Python")
        
        return [
            PromptMessage(
                role="user",
                content=f"""
请审查以下 {language} 代码:

```{language.lower()}
{code}
```

请关注:
1. 代码规范
2. 潜在 bug
3. 性能问题
4. 安全漏洞
5. 改进建议
"""
            )
        ]
    
    elif name == "write_blog":
        topic = arguments.get("topic", "")
        
        return [
            PromptMessage(
                role="user",
                content=f"""
请撰写一篇关于「{topic}」的技术博客。

要求:
1. 2000 字左右
2. 包含代码示例
3. 面向初学者
4. 结构清晰(为什么 → 是什么 → 怎么做 → 总结)
5. 使用中文
"""
            )
        ]
    
    return []

print("✅ Prompt Server 实现完成")
print("\nPrompt 模板的好处:")
print("  - 标准化常见任务")
print("  - 提升 Prompt 质量")
print("  - 便于团队协作")

## 实验 4: 完整的 MCP Server 示例

结合 Tools、Resources、Prompts 的完整示例。

In [ ]:
# 完整的 MCP Server 代码
# 保存为 demo_server.py

complete_server_code = '''
from mcp.server import Server
from mcp.server.stdio import stdio_server
from mcp.types import Tool, Resource, Prompt, TextContent, PromptMessage
import asyncio

# 创建 Server
app = Server("demo-server")

# === Tools ===
@app.list_tools()
async def list_tools() -> list[Tool]:
    return [
        Tool(
            name="echo",
            description="回显输入的文本",
            inputSchema={
                "type": "object",
                "properties": {
                    "message": {"type": "string", "description": "要回显的消息"}
                },
                "required": ["message"]
            }
        )
    ]

@app.call_tool()
async def call_tool(name: str, arguments: dict) -> list[TextContent]:
    if name == "echo":
        message = arguments.get("message", "")
        return [TextContent(type="text", text=f"Echo: {message}")]
    return [TextContent(type="text", text="未知工具")]

# === Resources ===
@app.list_resources()
async def list_resources() -> list[Resource]:
    return [
        Resource(
            uri="file:///readme",
            name="README",
            description="项目说明文档",
            mimeType="text/markdown"
        )
    ]

@app.read_resource()
async def read_resource(uri: str) -> str:
    if uri == "file:///readme":
        return "# Demo MCP Server\\n\\n这是一个示例 MCP Server。"
    return "资源不存在"

# === Prompts ===
@app.list_prompts()
async def list_prompts() -> list[Prompt]:
    return [
        Prompt(
            name="hello",
            description="打招呼模板",
            arguments=[]
        )
    ]

@app.get_prompt()
async def get_prompt(name: str, arguments: dict) -> list[PromptMessage]:
    if name == "hello":
        return [
            PromptMessage(
                role="user",
                content="请用友好的方式打招呼,并介绍你自己。"
            )
        ]
    return []

# === 启动 ===
async def main():
    async with stdio_server() as (read_stream, write_stream):
        await app.run(
            read_stream,
            write_stream,
            app.create_initialization_options()
        )

if __name__ == "__main__":
    asyncio.run(main())
'''

# 保存到文件
with open("demo_server.py", "w", encoding="utf-8") as f:
    f.write(complete_server_code)

print("✅ 完整 MCP Server 已保存到 demo_server.py")
print("\n配置 Claude Desktop:")
print("编辑 ~/Library/Application Support/Claude/claude_desktop_config.json")
print("""{
  "mcpServers": {
    "demo": {
      "command": "python",
      "args": ["/path/to/demo_server.py"]
    }
  }
}""")

## 实验 5: MCP 协议交互演示

模拟 MCP Client 和 Server 的交互过程。

In [ ]:
import json

# 模拟 MCP 协议交互

print("=== MCP 协议交互示例 ===")
print("\n1️⃣ Client 请求工具列表")

request_1 = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/list",
    "params": {}
}
print(f"请求: {json.dumps(request_1, indent=2, ensure_ascii=False)}")

response_1 = {
    "jsonrpc": "2.0",
    "id": 1,
    "result": {
        "tools": [
            {
                "name": "add",
                "description": "计算两个数的和",
                "inputSchema": {
                    "type": "object",
                    "properties": {
                        "a": {"type": "number"},
                        "b": {"type": "number"}
                    },
                    "required": ["a", "b"]
                }
            }
        ]
    }
}
print(f"\n响应: {json.dumps(response_1, indent=2, ensure_ascii=False)}")

print("\n\n2️⃣ Client 调用工具")

request_2 = {
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/call",
    "params": {
        "name": "add",
        "arguments": {
            "a": 123,
            "b": 456
        }
    }
}
print(f"请求: {json.dumps(request_2, indent=2, ensure_ascii=False)}")

response_2 = {
    "jsonrpc": "2.0",
    "id": 2,
    "result": {
        "content": [
            {
                "type": "text",
                "text": "579"
            }
        ]
    }
}
print(f"\n响应: {json.dumps(response_2, indent=2, ensure_ascii=False)}")

print("\n\n3️⃣ Client 读取资源")

request_3 = {
    "jsonrpc": "2.0",
    "id": 3,
    "method": "resources/read",
    "params": {
        "uri": "file:///readme"
    }
}
print(f"请求: {json.dumps(request_3, indent=2, ensure_ascii=False)}")

response_3 = {
    "jsonrpc": "2.0",
    "id": 3,
    "result": {
        "contents": [
            {
                "uri": "file:///readme",
                "mimeType": "text/markdown",
                "text": "# README\n\n项目说明"
            }
        ]
    }
}
print(f"\n响应: {json.dumps(response_3, indent=2, ensure_ascii=False)}")

## 动手练习

1. **实现一个文件系统 MCP Server**: 提供文件读取、写入、列表等工具
2. **实现一个数据库 MCP Server**: 提供 SQL 查询工具
3. **实现一个 HTTP MCP Server**: 提供 GET/POST 请求工具
4. **集成到 Claude Desktop**: 将你的 MCP Server 配置到 Claude Desktop 并测试

---

## 关键要点总结

1. **MCP Server 提供三种能力**: Tools、Resources、Prompts
2. **协议基于 JSON-RPC 2.0**: 简单、清晰、易于实现
3. **传输层支持 stdio 和 SSE**: 本地和远程都可以
4. **实现简单**: 官方 SDK 提供了完善的抽象
5. **生态丰富**: Claude Desktop、Cursor、Windsurf 等原生支持

---

**下一步**: 学习 [11.2 协议对比](./protocols_overview.ipynb),对比 MCP、A2A、ANP 三大协议。